In [3]:
!pip install transformers
from transformers import TrainingArguments

class DistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
    super().__init__(*args,**kwargs)
    self.alpha = alpha
    self.temperature = temperature

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.3 MB/s eta 0:00:00


In [4]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

class DistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher_model = teacher_model

  def compute_loss(self,model,inputs,return_outputs=False):
    student_outputs = model(**inputs)
    cross_ent = student_outputs.loss
    student_logits = student_outputs.logits

    with torch.no_grad():
      teacher_outputs = self.teacher_model(**inputs)
      teacher_logits = teacher_outputs.logits

    # Probabilities softening and distillation loss
    function_loss = nn.KLDivLoss(reduction="batchmean")
    loss_kd = self.args.temperature ** 2 * function_loss(
        F.log_softmax(student_logits / self.args.temperature,dim = -1),
        F.softmax(teacher_logits / self.args.temperature,dim=-1)
    )
    loss = self.args.alpha * cross_ent + (1 - self.args.alpha) * loss_kd
    return (loss, student_outputs) if return_outputs else loss

